In [1]:
# 1. 라이브러리 import
import random # random 수 생성
import pandas as pd # 데이터 처리/분석
import numpy as np # 행렬, 배열 연산
import os # 운영체제 관련 기능 사용
import cv2 # 이미지 처리

# 사이킷런
from sklearn import preprocessing # 데이터 전처리
from sklearn.model_selection import train_test_split # 데이터 분할

# 파이토치
import torch
import torch.nn as nn # 신경망
import torch.optim as optim # 최적화
import torch.nn.functional as F # 신경망 연산
from torch.utils.data import Dataset, DataLoader # 데이터 사용 (Dataset - 샘플과 정답(label)을 저장, DataLoader - Dataset 을 샘플에 쉽게 접근할 수 있도록)

from tqdm.auto import tqdm # 진행바 깔끔 출력

import albumentations as A # 이미지 증강 ### https://hoya012.github.io/blog/albumentation_tutorial/ 다른 이미지 증강법 쓰면?
from albumentations.pytorch.transforms import ToTensorV2 # 파이토치 호환

import torchvision.models as models # pretrained 모델 사용 (efficientnet)

from sklearn.metrics import f1_score # 모델 평가 지표인 f1 사용

import warnings # 경고 메시지 관리
warnings.filterwarnings(action='ignore') # 경고 메시지 무시
import torch, gc # 메모리 관리 위해 PyTorch와 가비지 컬렉터 import

from torch.optim import lr_scheduler # 학습률 스케줄러

!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=c095cd235b8f77b7949de326de70cd456f322d29c1bb2dac17f429909687c50e
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [2]:
# 디바이스 설정 (gpu 가능 시 cuda 장치, 안되면 cpu)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
# 설정 (하이퍼파라미터 설정)
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':30,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [4]:
def seed_everything(seed):
  random.seed(seed) # random 초기화해 시드값에 따라 같은 random 수 생성
  os.environ['PYTHONHASHSEED'] = str(seed) # 파이썬 해시 시드를 설정하여 해시 기반 작업의 일관성을 유지
  np.random.seed(seed) # numpy 초기화해 시드값에 따라 같은 random 수 생성
  torch.manual_seed(seed) # 파이토치의 random 초기화해 시드값에 따라 같은 random 수 생성
  torch.cuda.manual_seed(seed) # cuda(GPU)의 random 초기화해 시드값에 따라 같은 random 수 생성
  torch.backends.cudnn.deterministic = True # CuDNN 라이브러리의 동일한 연산을 보장
  torch.backends.cudnn.benchmark = True # CuDNN 연산 최적화 사용

# seed 고정
seed_everything(CFG['SEED'])

In [6]:
%cd "/content/"
!unzip "/content/drive/MyDrive/open.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: open/train/4097.jpg     
  inflating: __MACOSX/open/train/._4097.jpg  
  inflating: open/train/3920.jpg     
  inflating: __MACOSX/open/train/._3920.jpg  
  inflating: open/train/5389.jpg     
  inflating: __MACOSX/open/train/._5389.jpg  
  inflating: open/train/3908.jpg     
  inflating: __MACOSX/open/train/._3908.jpg  
  inflating: open/train/4901.jpg     
  inflating: __MACOSX/open/train/._4901.jpg  
  inflating: open/train/1879.jpg     
  inflating: __MACOSX/open/train/._1879.jpg  
  inflating: open/train/4915.jpg     
  inflating: __MACOSX/open/train/._4915.jpg  
  inflating: open/train/0352.jpg     
  inflating: __MACOSX/open/train/._0352.jpg  
  inflating: open/train/4134.jpg     
  inflating: __MACOSX/open/train/._4134.jpg  
  inflating: open/train/2545.jpg     
  inflating: __MACOSX/open/train/._2545.jpg  
  inflating: open/train/3883.jpg     
  inflating: __MACOSX/open/train/._3883.jpg  
  inflating: open/train/2223.jpg     
  

In [8]:
import glob
filepaths1 = list(glob.glob('/content/open/test/*.jpg'))
filepaths2 = list(glob.glob('/content/open/train/*.jpg'))
print(len(filepaths1))
print(len(filepaths2))

12670
5911


In [9]:
# 2. 파이토치 데이터셋 생성

# 현재 작업디렉토리 return
base_dir = os.getcwd() # 기본 디렉토리 경로
data_dir = '/content/open' # 데이터 파일이 있는 디렉토리 경로

df = pd.read_csv(f'{data_dir}/train.csv')
df1 = pd.read_csv(f'{data_dir}/train.csv')

df.head()

,id,img_path,artist
0,0,./train/0000.jpg,Diego Velazquez
1,1,./train/0001.jpg,Vincent van Gogh
2,2,./train/0002.jpg,Claude Monet
3,3,./train/0003.jpg,Edgar Degas
4,4,./train/0004.jpg,Hieronymus Bosch


In [10]:
# Label Encoding
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values) # artist를 정수 형태로

df.head()

,id,img_path,artist
0,0,./train/0000.jpg,9
1,1,./train/0001.jpg,48
2,2,./train/0002.jpg,7
3,3,./train/0003.jpg,10
4,4,./train/0004.jpg,24


In [11]:
# 데이터 train, test 나누기 (데이터의 20%를 testset)
train_df, val_df, _, _ = train_test_split(df, df['artist'].values, test_size=0.2, random_state=CFG['SEED'])

In [12]:
# 'train_df'를 'id' 열 기준으로 정렬
train_df = train_df.sort_values(by=['id'])
train_df.head()

,id,img_path,artist
0,0,./train/0000.jpg,9
2,2,./train/0002.jpg,7
3,3,./train/0003.jpg,10
5,5,./train/0005.jpg,38
6,6,./train/0006.jpg,43


In [13]:
# val_df'를 'id' 열 기준으로 정렬
val_df = val_df.sort_values(by=['id'])
val_df.head()

,id,img_path,artist
1,1,./train/0001.jpg,48
4,4,./train/0004.jpg,24
17,17,./train/0017.jpg,10
21,21,./train/0021.jpg,29
29,29,./train/0029.jpg,28


In [14]:
def get_data(df, infer=False):
  if infer:
    return df['img_path'].values # img_path return
  return df['img_path'].values, df['artist'].values # img_path, artist return

train_img_paths, train_labels = get_data(train_df) # train
val_img_paths, val_labels = get_data(val_df) # test

print(f"{data_dir}/{train_img_paths[0].split('/')[-2]}/{train_img_paths[0].split('/')[-1]}")

/content/open/train/0000.jpg


In [15]:
class CustomDataset(Dataset):
  def __init__(self, img_paths, labels, transforms=None):
    self.img_paths = img_paths
    self.labels = labels
    self.transforms = transforms

  def __getitem__(self, index):
    img_path = self.img_paths[index]
    image = cv2.imread(f"{data_dir}/{img_path.split('/')[-2]}/{img_path.split('/')[-1]}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    if self.transforms is not None:
      image = self.transforms(image=image)['image'] # 이미지 변환

    if self.labels is not None:
      label = self.labels[index]
      return image, label
    else:
      return image

  # 총 데이터 수
  def __len__(self):
    return len(self.img_paths)

# 이미지 크기 조정
def resize_transform(size, state='train'):
  if state == 'train':
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomResizedCrop(height=size, width=size, scale=(0.3,1.0)),
        A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
  else:
    transform = A.Compose([
        A.Resize(size, size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
        ToTensorV2()
    ])

  return transform
# 256, 288, 320, 384, 456, 528
# 240, 260, 300, 380, 456, 528

In [16]:
# 3. 파이토치 모델 만들기
class BaseModel(nn.Module): # 모델이 224 픽셀 크기의 이미지에 사용될 것 # torch.nn.Module 은 PyTorch의 모든 Neural Network의 Base Class
  def __init__(self, num_classes=len(le.classes_)):
    super(BaseModel, self).__init__() # 모델 초기화
    self.backbone = models.efficientnet_b0(pretrained=True) # 이미지 특성 추출

    self.classifier = nn.Sequential(nn.ReLU(), # EfficientNet의 출력 특성을 받아서 최종 클래스 예측을 수행
                                    nn.Dropout(),
                                    nn.Linear(1000, num_classes))

  def forward(self, x):
    x = self.backbone(x)
    x = self.classifier(x)
    return x # 최종 클래스 예측 결과

class EfficientNet_B1(nn.Module): # 256
  def __init__(self, num_classes=len(le.classes_)):
    super(EfficientNet_B1, self).__init__()
    self.backbone = models.efficientnet_b1(pretrained=True)

    self.classifier = nn.Sequential(nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(1000, num_classes))

  def forward(self, x):
    x = self.backbone(x)
    x = self.classifier(x)
    return x

class EfficientNet_B2(nn.Module): # 288
  def __init__(self, num_classes=len(le.classes_)):
    super(EfficientNet_B2, self).__init__()
    self.backbone = models.efficientnet_b2(pretrained=True)

    self.classifier = nn.Sequential(nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(1000, num_classes))

  def forward(self, x):
    x = self.backbone(x)
    x = self.classifier(x)
    return x

class EfficientNet_B3(nn.Module): # 320
  def __init__(self, num_class=len(le.classes_)):
    super(EfficientNet_B3, self).__init__()
    self.backbone = models.efficientnet_b3(pretrained=True)

    self.classifier = nn.Sequential(nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(1000, num_classes))

  def forward(self, x):
    x = self.backbone(x)
    x = self.classifier(x)
    return x

class EfficientNet_B4(nn.Module): # 384
  def __init__(self, num_classes=len(le.classes_)):
    super(EfficientNet_B4, self).__init__()
    self.backbone = models.efficientnet_b4(pretrained=True)

    self.classifier = nn.Sequential(nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(1000, num_classes))

  def forward(self, x):
    x = self.backbone(x)
    x = self.classifier(x)
    return x

class EfficientNet_B5(nn.Module): # 456
  def __init__(self, num_classes=len(le.classes_)):
    super(EfficientNet_B5, self).__init__()
    self.backbone = models.efficientnet_b5(pretrained=True)

    self.classifier = nn.Sequential(nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(1000, num_classes))

  def forward(self, x):
    x = self.backbone(x)
    x = self.classifier(x)
    return x

class EfficientNet_B6(nn.Module):
  def __init__(self, num_classes=len(le.classes_)):
    super(EfficientNet_B6, self).__init__()
    self.backbone = models.efficientnet_b6(pretrained=True)

    self.classifier = nn.Sequential(nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(1000, num_classes))

  def forawrd(self, x):
    x = self.backbone(x)
    x = self.classifier(x)
    return x

class EfficientNet_Bn(nn.Module):
    def __init__(self, num_classes=len(le.classes_), b_num=0):
        super(EfficientNet_Bn, self).__init__()
        self.backbone = EfficientNet.from_pretrained(f'efficientnet-b{b_num}')

        self.classifier = nn.Sequential(
                                        nn.ReLU(),
                                        nn.Dropout(),
                                        nn.Linear(1000, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [20]:
# 4. 모델 Train
def train(model, epochs, optimizer, train_loader, test_loader, scheduler, device):
  model.to(device) # 모델을 지정한 장치 (CPU 또는 GPU)로 이동

  criterion = nn.CrossEntropyLoss().to(device) # 손실 함수 (CrossEntropyLoss)

  best_score = 0 # 현재까지 관찰된 가장 높은 검증 성능
  best_model = None # 현재까지 관찰된 최고 성능을 달성한 모델

  for epoch in range(1, epochs+1):
    model.train() # 모델을 학습 모드로 설정
    train_loss = [] # 각 에폭에서 학습 데이터에 대한 손실을 추적
    # 학습 데이터셋을 순회하며 모델을 학습
    for img, label in tqdm(iter(train_loader)):
      img, label = img.float().to(device), label.type(torch.LongTensor).to(device) # 이미지와 레이블을 지정한 장치로 이동

      optimizer.zero_grad() # 기울기 초기화

      model_pred = model(img) # 모델의 예측 생성

      loss = criterion(model_pred, label) # 손실 계산

      loss.backward() # 역전파 및 가중치 업데이트
      optimizer.step()

      train_loss.append(loss.item()) # 학습 손실 저장

    tr_loss = np.mean(train_loss) # 학습 손실 평균 계산

    val_loss, val_score = validation(model, criterion, test_loader, device) # 검증 손실과 F1 점수 계산

    print(f'Epoch [{epoch}], Train Loss: [{tr_loss:.5f}] Val Loss: [{val_loss:.5f}] Val F1 Score: [{val_score:.5f}]')  # 에폭별 학습 및 검증 결과 출력

    if scheduler is not None: # 스케줄러가 주어진 경우 학습률 스케줄링 적용
      scheduler.step()

    if best_score < val_score: # 현재까지의 최고 성능을 기록한 경우, 모델 저장
      best_model = model
      best_score = val_score

  return best_model # 최고 성능을 달성한 모델 반환

def competition_metric(true, pred):
  return f1_score(true, pred, average="macro") # F1 점수를 계산하여 반환

def validation(model, criterion, test_loader, device):
  model.eval() # 모델을 평가 모드로 설정

  # 모델의 예측과 실제 레이블을 저장할 리스트
  model_preds = []
  true_labels = []

  val_loss = [] # 검증 손실 저장

  with torch.no_grad():
    # 검증 데이터셋을 순회하며 모델 평가
    for img, label in tqdm(iter(test_loader)):
      img, label = img.float().to(device), label.type(torch.LongTensor).to(device) # 이미지와 레이블을 지정한 장치로 이동

      model_pred = model(img) # 모델의 예측 생성

      # 손실 계산
      loss = criterion(model_pred, label)
      val_loss.append(loss.item())

      # 모델의 예측과 실제 레이블 저장
      model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
      true_labels += label.detach().cpu().numpy().tolist()

  # 검증 데이터의 F1 점수 계산
  val_f1 = competition_metric(true_labels, model_preds)

  return np.mean(val_loss), val_f1

num_epoch = 30

In [21]:
train_dataset_0 = CustomDataset(train_img_paths, train_labels, resize_transform(224,'train')) # trainset 처리하기 위한 사용자 정의 데이터셋 생성
train_loader_0 = DataLoader(train_dataset_0, batch_size=32, shuffle=True, num_workers=0) # 데이터를 배치 단위로 로드

val_dataset_0 = CustomDataset(val_img_paths, val_labels, resize_transform(224,'test')) # testset 처리하기 위한 사용자 정의 데이터셋 생성
val_loader_0 = DataLoader(val_dataset_0, batch_size=32, shuffle=False, num_workers=0) # 데이터를 배치 단위로 로드

model_0 = EfficientNet_Bn(b_num=0) # 모델 초기화
optimizer_0 = torch.optim.Adam(params = model_0.parameters(), lr = CFG["LEARNING_RATE"]) # 모델 가중치 업데이트하는 옵티마이저
scheduler_0 = lr_scheduler.MultiStepLR(optimizer_0, milestones=[10,20], gamma=0.5) # 학습률 스케줄링을 수행하는 스케줄러
infer_model_0 = train(model_0, num_epoch, optimizer_0, train_loader_0, val_loader_0, scheduler_0, device) # 모델을 학습하고 최상의 성능을 달성한 모델

Loaded pretrained weights for efficientnet-b0


  0%|          | 0/148 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
torch.save(infer_model_0, 'weights/b0_32_30.pt') # 학습된 모델의 상태를 파일에 저장
gc.collect() # 메모리 관리
torch.cuda.empty_cache() # GPU 메모리 관리

In [32]:
#0.76917 batch:16, epoch:30, LR:[10,20]
#0.73733 batch:8, epoch:30, LR:[9,19,27]
train_dataset_1 = CustomDataset(train_img_paths, train_labels, resize_transform(256,'train'))
train_loader_1 = DataLoader(train_dataset_1, batch_size=32, shuffle=True, num_workers=0)

val_dataset_1 = CustomDataset(val_img_paths, val_labels, resize_transform(256,'test'))
val_loader_1 = DataLoader(val_dataset_1, batch_size=32, shuffle=False, num_workers=0)

model_1 = EfficientNet_Bn(b_num=1)
optimizer_1 = torch.optim.Adam(params = model_1.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_1 = lr_scheduler.MultiStepLR(optimizer_1, milestones=[10,20], gamma=0.5)
infer_model_1 = train(model_1, num_epoch, optimizer_1, train_loader_1, val_loader_1, scheduler_1, device)

Loaded pretrained weights for efficientnet-b1


  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
torch.save(infer_model_1, 'weights/b1_32_39.pt')
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#0.78464 batch:16, epoch:30, LR:[10,20]
#0.77673 batch:8, epoch:30, LR:[9,18,27]
train_dataset_2 = CustomDataset(train_img_paths, train_labels, resize_transform(288,'train'))
train_loader_2 = DataLoader(train_dataset_2, batch_size=32, shuffle=True, num_workers=0)

val_dataset_2 = CustomDataset(val_img_paths, val_labels, resize_transform(299,'test'))
val_loader_2 = DataLoader(val_dataset_2, batch_size=32, shuffle=False, num_workers=0)

model_2 = EfficientNet_Bn(b_num=2)
optimizer_2 = torch.optim.Adam(params = model_2.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_2 = lr_scheduler.MultiStepLR(optimizer_2, milestones=[10,20], gamma=0.5)
infer_model_2 = train(model_2, num_epoch, optimizer_2, train_loader_2, val_loader_2, scheduler_2, device)

In [ ]:
torch.save(infer_model_2, 'weights/b2_32_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 0.77395 batch:32, epoch:30, LR: [10,20]
# 0.77227 batch:16, epcoh:30, lr:[10,20]
# 0.76952 batch:8, epoch:30, LR: [9,18,27]
train_dataset_3 = CustomDataset(train_img_paths, train_labels, resize_transform(320,'train'))
train_loader_3 = DataLoader(train_dataset_3, batch_size=32, shuffle=True, num_workers=0)

val_dataset_3 = CustomDataset(val_img_paths, val_labels, resize_transform(320,'test'))
val_loader_3 = DataLoader(val_dataset_3, batch_size=32, shuffle=False, num_workers=0)

model_3 = EfficientNet_Bn(b_num=3)
optimizer_3 = torch.optim.Adam(params = model_3.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_3 = lr_scheduler.MultiStepLR(optimizer_3, milestones=[10,20], gamma=0.5)
infer_model_3 = train(model_3, num_epoch, optimizer_3, train_loader_3, val_loader_3, scheduler_3, device)

In [ ]:
torch.save(infer_model_3, 'weights/b3_32_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#0.79137 batch:16, epoch:30, lr[10,20]
#0.79599 batch:8, epoch:30, LR: [9,18,27]
train_dataset_4 = CustomDataset(train_img_paths, train_labels, resize_transform(384,'train'))
train_loader_4 = DataLoader(train_dataset_4, batch_size=16, shuffle=True, num_workers=0)

val_dataset_4 = CustomDataset(val_img_paths, val_labels, resize_transform(384,'test'))
val_loader_4 = DataLoader(val_dataset_4, batch_size=16, shuffle=False, num_workers=0)

model_4 = EfficientNet_Bn(b_num=4)
optimizer_4 = torch.optim.Adam(params = model_4.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_4 = lr_scheduler.MultiStepLR(optimizer_4, milestones=[10,20], gamma=0.5)
infer_model_4 = train(model_4, num_epoch, optimizer_4, train_loader_4, val_loader_4, scheduler_4, device)

In [ ]:
torch.save(infer_model_4,'weights/b4_16_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#0.82004 batch:8, epoch : 30, lr:[9,18,27]
train_dataset_5 = CustomDataset(train_img_paths, train_labels, resize_transform(456,'train'))
train_loader_5 = DataLoader(train_dataset_5, batch_size=8, shuffle=True, num_workers=0)

val_dataset_5 = CustomDataset(val_img_paths, val_labels, resize_transform(456,'test'))
val_loader_5 = DataLoader(val_dataset_5, batch_size=8, shuffle=False, num_workers=0)

model_5 = EfficientNet_Bn(b_num=5)
optimizer_5 = torch.optim.Adam(params = model_5.parameters(), lr = CFG["LEARNING_RATE"])
scheduler_5 = lr_scheduler.MultiStepLR(optimizer_5, milestones=[10,20], gamma=0.5)
infer_model_5 = train(model_5, num_epoch, optimizer_5, train_loader_5, val_loader_5, scheduler_5, device)

In [ ]:
torch.save(infer_model_5,'weights/b5_8_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#0.80073 batch:4, epoch:30, lr:[8,16,24]
train_dataset_6 = CustomDataset(train_img_paths, train_labels, resize_transform(528,'train'))
train_loader_6 = DataLoader(train_dataset_6, batch_size=4, shuffle=True, num_workers=0)

val_dataset_6 = CustomDataset(val_img_paths, val_labels, resize_transform(528,'test'))
val_loader_6 = DataLoader(val_dataset_6, batch_size=4, shuffle=False, num_workers=0)

model_6 = EfficientNet_Bn(b_num=6)
optimizer_6 = torch.optim.Adam(params = model_6.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = lr_scheduler.MultiStepLR(optimizer_6, milestones=[10,20], gamma=0.5)
infer_model_6 = train(model_6,num_epoch, optimizer_6, train_loader_6, val_loader_6, scheduler, device)

In [ ]:
torch.save(infer_model_6,'weights/b6_4_30.pt')
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 5.모델 Test
test_df = pd.read_csv(f'{data_dir}/test.csv')
test_df.head()

In [ ]:
test_img_paths = get_data(test_df, infer=True)

test_dataset_1 = CustomDataset(test_img_paths, None, resize_transform(256,'test'))
test_loader_1 = DataLoader(test_dataset_1, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

test_dataset_2 = CustomDataset(test_img_paths, None, resize_transform(288,'test'))
test_loader_2 = DataLoader(test_dataset_2, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

test_dataset_3 = CustomDataset(test_img_paths, None, resize_transform(320,'test'))
test_loader_3 = DataLoader(test_dataset_3, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

test_dataset_4 = CustomDataset(test_img_paths, None, resize_transform(384,'test'))
test_loader_4 = DataLoader(test_dataset_4, batch_size=16, shuffle=False, num_workers=0)

test_dataset_5 = CustomDataset(test_img_paths, None, resize_transform(456,'test'))
test_loader_5 = DataLoader(test_dataset_5, batch_size=16, shuffle=False, num_workers=0)

test_dataset_6 = CustomDataset(test_img_paths, None, resize_transform(528,'test'))
test_loader_6 = DataLoader(test_dataset_6, batch_size=8, shuffle=False, num_workers=0)

In [ ]:
# Inference Test
def inference(model, test_loader, device):
  model.to(device)
  model.eval()

  model_preds = []

  with torch.no_grad():
    for img in tqdm(iter(test_loader)):
      img = img.float().to(device)

      model_pred = model(img)
      model_preds += model_pred.detach().cpu().numpy().tolist()

  print('Done.')
  return model_preds

#preds = inference(infer_model_5, test_loader_5, device)